# Scraping Transfermarkt

In [90]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Functions

In [116]:
def top_n_leagues(n):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    page = "https://www.transfermarkt.co.uk/wettbewerbe/europa"
    pageTree = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    LeagueList = []
    League = pageSoup.find_all("img", {"class": "continental-league-emblem"})

    if n > len(League):
        print("There is not that many leagues")
    else:
        for i in range(0,n):
            LeagueList.append(str(League[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return LeagueList

In [117]:
def top_n_leagues_links(n):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    page = "https://www.transfermarkt.co.uk/wettbewerbe/europa"
    pageTree = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    LeagueList = []

    for name in n:
        League = pageSoup.find_all("a", {"title": name})
        for i in range(0,len(League)):
            LeagueList.append("https://www.transfermarkt.co.uk" + str(League[i]).split('" title', 1)[0].split('href="', 1)[1])

    return LeagueList

In [147]:
# Prints names only
def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList

In [119]:
# Prints link only
def team_links(links, team_names):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        for name in team_names:
            Team = pageSoup.find_all("a", {"title": name})
            x = 0
            for i in range(0,len(Team)):
                TeamList.append("https://www.transfermarkt.co.uk" + str(Team[i]).split('" title',1)[0].split('href="', 1)[1])
                x += 1
                if x > 0:
                    break

    return TeamList

In [120]:
def build_df(links):

    PlayersList = []
    AgeList=[]
    NationList=[]
    ValuesList = []
    PositionsList=[]
    ClubList = []

    for a in links:
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
        page = a
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Players = pageSoup.find_all("img", {"class": "bilderrahmen-fixed lazy lazy"})
        
        Values = pageSoup.find_all("td", {"class": "rechts hauptlink"})

        for i in range(0,len(Values)):
            ValuesList.append(Values[i].text)

        for i in range(0,len(Players)):
            #print(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1], Values[i].text)
            PlayersList.append(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1])

    df= pd.DataFrame({"Players":PlayersList, "Values":ValuesList,})


    df.loc[(df.Values == '\xa0'),'Values']= None
        
    return df

# Premier League

In [96]:
leagues = top_n_leagues(1)
leagues

['Premier League']

In [97]:
league_links = top_n_leagues_links(leagues)
league_links

['https://www.transfermarkt.co.uk/premier-league/startseite/wettbewerb/GB1']

In [98]:
team_names = team_names(league_links)
team_names

['Manchester City',
 'Arsenal FC',
 'Chelsea FC',
 'Liverpool FC',
 'Manchester United',
 'Tottenham Hotspur',
 'Newcastle United',
 'Aston Villa',
 'Brighton &amp; Hove Albion',
 'West Ham United',
 'Brentford FC',
 'Wolverhampton Wanderers',
 'Everton FC',
 'Nottingham Forest',
 'Crystal Palace',
 'AFC Bournemouth',
 'Fulham FC',
 'Burnley FC',
 'Sheffield United',
 'Luton Town',
 'Arsenal FC',
 'Newcastle United',
 'Brentford FC',
 'Burnley FC',
 'Luton Town',
 'AFC Bournemouth',
 'Manchester United',
 'Fulham FC',
 'Crystal Palace',
 'Nottingham Forest',
 'Everton FC',
 'Chelsea FC',
 'Wolverhampton Wanderers',
 'Aston Villa',
 'West Ham United',
 'Sheffield United',
 'Manchester City',
 'Tottenham Hotspur',
 'Liverpool FC',
 'Brighton &amp; Hove Albion']

In [102]:
every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = [s[:-1] + '1' for s in every_team_links]
every_team_links = every_team_links[0:20]
every_team_links

['https://www.transfermarkt.co.uk/manchester-city/startseite/verein/281/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-arsenal/startseite/verein/11/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-chelsea/startseite/verein/631/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-liverpool/startseite/verein/31/saison_id/2021',
 'https://www.transfermarkt.co.uk/manchester-united/startseite/verein/985/saison_id/2021',
 'https://www.transfermarkt.co.uk/tottenham-hotspur/startseite/verein/148/saison_id/2021',
 'https://www.transfermarkt.co.uk/newcastle-united/startseite/verein/762/saison_id/2021',
 'https://www.transfermarkt.co.uk/aston-villa/startseite/verein/405/saison_id/2021',
 'https://www.transfermarkt.co.uk/west-ham-united/startseite/verein/379/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-brentford/startseite/verein/1148/saison_id/2021',
 'https://www.transfermarkt.co.uk/wolverhampton-wanderers/startseite/verein/543/saison_id/2021',
 'https://www.transfermarkt.c

In [107]:
df = build_df(every_team_links)

In [109]:
df = df.dropna()
df

,Players,Values
0,Ederson,€45.00m
1,Zack Steffen,€6.00m
2,Scott Carson,€300k
4,Rúben Dias,€75.00m
5,Aymeric Laporte,€45.00m
...,...,...
797,Eddie Nketiah,€16.00m
798,Pierre-Emerick Aubameyang,€15.00m
799,Alexandre Lacazette,€15.00m
800,Folarin Balogun,€4.00m


In [110]:
df.to_csv('PL2021.csv', index=False)

# La Liga

In [150]:
leagues = top_n_leagues(5)
leagues = leagues[1:-3]
leagues

['LaLiga']

In [151]:
league_links = top_n_leagues_links(leagues)
league_links

['https://www.transfermarkt.co.uk/laliga/startseite/wettbewerb/ES1']

In [152]:
# Prints names only
def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList

team_names = team_names(league_links)
team_names

['Real Madrid',
 'FC Barcelona',
 'Atlético de Madrid',
 'Real Sociedad',
 'Villarreal CF',
 'Real Betis Balompié',
 'Sevilla FC',
 'Athletic Bilbao',
 'Valencia CF',
 'CA Osasuna',
 'Celta de Vigo',
 'Getafe CF',
 'UD Almería',
 'Girona FC',
 'RCD Mallorca',
 'Rayo Vallecano',
 'Cádiz CF',
 'Granada CF',
 'UD Las Palmas',
 'Deportivo Alavés',
 'Atlético de Madrid',
 'Celta de Vigo',
 'Girona FC',
 'Getafe CF',
 'UD Almería',
 'Cádiz CF',
 'Deportivo Alavés',
 'Villarreal CF',
 'Valencia CF',
 'Real Sociedad',
 'FC Barcelona',
 'Athletic Bilbao',
 'UD Las Palmas',
 'Real Madrid',
 'Sevilla FC',
 'Rayo Vallecano',
 'CA Osasuna',
 'RCD Mallorca',
 'Real Betis Balompié',
 'Granada CF']

In [139]:
every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = [s[:-1] + '1' for s in every_team_links]
every_team_links = every_team_links[0:20]
every_team_links

['https://www.transfermarkt.co.uk/real-madrid/startseite/verein/418/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-barcelona/startseite/verein/131/saison_id/2021',
 'https://www.transfermarkt.co.uk/atletico-madrid/startseite/verein/13/saison_id/2021',
 'https://www.transfermarkt.co.uk/real-sociedad-san-sebastian/startseite/verein/681/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-villarreal/startseite/verein/1050/saison_id/2021',
 'https://www.transfermarkt.co.uk/real-betis-sevilla/startseite/verein/150/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-sevilla/startseite/verein/368/saison_id/2021',
 'https://www.transfermarkt.co.uk/athletic-bilbao/startseite/verein/621/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-valencia/startseite/verein/1049/saison_id/2021',
 'https://www.transfermarkt.co.uk/ca-osasuna/startseite/verein/331/saison_id/2021',
 'https://www.transfermarkt.co.uk/celta-vigo/startseite/verein/940/saison_id/2021',
 'https://www.transfermarkt.co.uk

In [140]:
df2 = build_df(every_team_links)

In [142]:
df2 = df2.dropna()
df2

,Players,Values
0,Thibaut Courtois,€60.00m
1,Andriy Lunin,€2.50m
2,Luis López,€300k
3,Toni Fuidias,€300k
5,Éder Militão,€60.00m
...,...,...
774,Mamadou Sylla,€1.20m
775,John Guidetti,€1.00m
776,Marc Tenas,€500k
777,Giorgi Gagua,€150k


In [143]:
df2.to_csv('La2021.csv', index=False)

# Serie A

In [158]:
leagues = top_n_leagues(5)
leagues = leagues[2:-2]
leagues

['Serie A']

In [159]:
league_links = top_n_leagues_links(leagues)
league_links

['https://www.transfermarkt.co.uk/serie-a/startseite/wettbewerb/IT1']

In [160]:
# Prints names only
def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList

team_names = team_names(league_links)
team_names

['SSC Napoli',
 'Inter Milan',
 'Juventus FC',
 'AC Milan',
 'Atalanta BC',
 'AS Roma',
 'SS Lazio',
 'ACF Fiorentina',
 'Udinese Calcio',
 'US Sassuolo',
 'Torino FC',
 'Bologna FC 1909',
 'AC Monza',
 'US Salernitana 1919',
 'US Lecce',
 'FC Empoli',
 'Hellas Verona',
 'Genoa CFC',
 'Cagliari Calcio',
 'Frosinone Calcio',
 'AC Milan',
 'FC Empoli',
 'US Sassuolo',
 'SSC Napoli',
 'AC Monza',
 'Cagliari Calcio',
 'Bologna FC 1909',
 'US Lecce',
 'Atalanta BC',
 'Juventus FC',
 'AS Roma',
 'ACF Fiorentina',
 'Torino FC',
 'Udinese Calcio',
 'SS Lazio',
 'US Salernitana 1919',
 'Hellas Verona',
 'Genoa CFC',
 'Inter Milan',
 'Frosinone Calcio']

In [161]:
every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = [s[:-1] + '1' for s in every_team_links]
every_team_links = every_team_links[0:20]
every_team_links

['https://www.transfermarkt.co.uk/ssc-napoli/startseite/verein/6195/saison_id/2021',
 'https://www.transfermarkt.co.uk/inter-mailand/startseite/verein/46/saison_id/2021',
 'https://www.transfermarkt.co.uk/juventus-fc/startseite/verein/506/saison_id/2021',
 'https://www.transfermarkt.co.uk/ac-mailand/startseite/verein/5/saison_id/2021',
 'https://www.transfermarkt.co.uk/atalanta-bergamo/startseite/verein/800/saison_id/2021',
 'https://www.transfermarkt.co.uk/as-rom/startseite/verein/12/saison_id/2021',
 'https://www.transfermarkt.co.uk/lazio-rom/startseite/verein/398/saison_id/2021',
 'https://www.transfermarkt.co.uk/ac-florenz/startseite/verein/430/saison_id/2021',
 'https://www.transfermarkt.co.uk/udinese-calcio/startseite/verein/410/saison_id/2021',
 'https://www.transfermarkt.co.uk/us-sassuolo/startseite/verein/6574/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-turin/startseite/verein/416/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-bologna/startseite/verein/1025/sa

In [162]:
df3 = build_df(every_team_links)

In [163]:
df3 = df3.dropna()
df3

,Players,Values
0,Alex Meret,€10.00m
1,David Ospina,€5.00m
2,Hubert Idasiak,€150k
3,Davide Marfella,€50k
5,Kalidou Koulibaly,€35.00m
...,...,...
822,Andrija Novakovich,€1.00m
823,Pietro Iemmello,€750k
824,Camillo Ciano,€500k
825,Alexander Satariano,€150k


In [164]:
df3.to_csv('SA2021.csv', index=False)

# Bundasliga

In [165]:
leagues = top_n_leagues(5)
leagues = leagues[3:-1]
leagues

['Bundesliga']

In [169]:
league_links = top_n_leagues_links(leagues)
league_links = league_links[0:1]
league_links

['https://www.transfermarkt.co.uk/bundesliga/startseite/wettbewerb/L1']

In [170]:
# Prints names only
def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList

team_names = team_names(league_links)
team_names

['Bayern Munich',
 'Bayer 04 Leverkusen',
 'Borussia Dortmund',
 'RB Leipzig',
 'Eintracht Frankfurt',
 'VfL Wolfsburg',
 'SC Freiburg',
 'Borussia Mönchengladbach',
 '1.FC Union Berlin',
 'FC Augsburg',
 'VfB Stuttgart',
 '1.FSV Mainz 05',
 'TSG 1899 Hoffenheim',
 'SV Werder Bremen',
 '1.FC Köln',
 'VfL Bochum',
 '1.FC Heidenheim 1846',
 'SV Darmstadt 98',
 '1.FC Köln',
 'VfL Wolfsburg',
 '1.FC Heidenheim 1846',
 'TSG 1899 Hoffenheim',
 'FC Augsburg',
 'SV Darmstadt 98',
 '1.FC Union Berlin',
 'SV Werder Bremen',
 'VfL Bochum',
 'Bayer 04 Leverkusen',
 'VfB Stuttgart',
 'SC Freiburg',
 '1.FSV Mainz 05',
 'Bayern Munich',
 'Eintracht Frankfurt',
 'Borussia Mönchengladbach',
 'Borussia Dortmund',
 'RB Leipzig']

In [171]:
every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = [s[:-1] + '1' for s in every_team_links]
every_team_links = every_team_links[0:18]
every_team_links

['https://www.transfermarkt.co.uk/bayern-munich/startseite/verein/27/saison_id/2021',
 'https://www.transfermarkt.co.uk/bayer-04-leverkusen/startseite/verein/15/saison_id/2021',
 'https://www.transfermarkt.co.uk/borussia-dortmund/startseite/verein/16/saison_id/2021',
 'https://www.transfermarkt.co.uk/rasenballsport-leipzig/startseite/verein/23826/saison_id/2021',
 'https://www.transfermarkt.co.uk/eintracht-frankfurt/startseite/verein/24/saison_id/2021',
 'https://www.transfermarkt.co.uk/vfl-wolfsburg/startseite/verein/82/saison_id/2021',
 'https://www.transfermarkt.co.uk/sc-freiburg/startseite/verein/60/saison_id/2021',
 'https://www.transfermarkt.co.uk/borussia-monchengladbach/startseite/verein/18/saison_id/2021',
 'https://www.transfermarkt.co.uk/1-fc-union-berlin/startseite/verein/89/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-augsburg/startseite/verein/167/saison_id/2021',
 'https://www.transfermarkt.co.uk/vfb-stuttgart/startseite/verein/79/saison_id/2021',
 'https://www.

In [172]:
df4 = build_df(every_team_links)

In [173]:
df4 = df4.dropna()
df4

,Players,Values
0,Manuel Neuer,€15.00m
1,Sven Ulreich,€900k
2,Christian Früchtl,€600k
3,Ron-Thorben Hoffmann,€500k
4,Lucas Hernández,€50.00m
...,...,...
631,Phillip Tietz,€2.00m
632,Luca Pfeiffer,€2.00m
633,Aaron Seydel,€600k
634,Braydon Manu,€550k


In [174]:
df4.to_csv('Bu2021.csv', index=False)

# Ligue 1

In [175]:
leagues = top_n_leagues(5)
leagues = leagues[4:]
leagues

['Ligue 1']

In [176]:
league_links = top_n_leagues_links(leagues)
league_links

['https://www.transfermarkt.co.uk/ligue-1/startseite/wettbewerb/FR1']

In [178]:
# Prints names only
def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList

team_names = team_names(league_links)
team_names

['Paris Saint-Germain',
 'Stade Rennais FC',
 'AS Monaco',
 'Olympique Marseille',
 'Olympique Lyon',
 'RC Lens',
 'OGC Nice',
 'LOSC Lille',
 'Montpellier HSC',
 'Stade Reims',
 'RC Strasbourg Alsace',
 'FC Nantes',
 'FC Lorient',
 'FC Toulouse',
 'Stade Brestois 29',
 'Clermont Foot 63',
 'FC Metz',
 'Le Havre AC',
 'AS Monaco',
 'Montpellier HSC',
 'Clermont Foot 63',
 'Stade Reims',
 'FC Lorient',
 'LOSC Lille',
 'Olympique Lyon',
 'FC Nantes',
 'RC Lens',
 'Olympique Marseille',
 'Le Havre AC',
 'RC Strasbourg Alsace',
 'Paris Saint-Germain',
 'OGC Nice',
 'FC Toulouse',
 'FC Metz',
 'Stade Rennais FC',
 'Stade Brestois 29']

In [179]:
every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = [s[:-1] + '1' for s in every_team_links]
every_team_links = every_team_links[0:18]
every_team_links

['https://www.transfermarkt.co.uk/paris-saint-germain/startseite/verein/583/saison_id/2021',
 'https://www.transfermarkt.co.uk/fc-stade-rennes/startseite/verein/273/saison_id/2021',
 'https://www.transfermarkt.co.uk/as-monaco/startseite/verein/162/saison_id/2021',
 'https://www.transfermarkt.co.uk/olympique-marseille/startseite/verein/244/saison_id/2021',
 'https://www.transfermarkt.co.uk/olympique-lyon/startseite/verein/1041/saison_id/2021',
 'https://www.transfermarkt.co.uk/rc-lens/startseite/verein/826/saison_id/2021',
 'https://www.transfermarkt.co.uk/ogc-nizza/startseite/verein/417/saison_id/2021',
 'https://www.transfermarkt.co.uk/losc-lille/startseite/verein/1082/saison_id/2021',
 'https://www.transfermarkt.co.uk/montpellier-hsc/startseite/verein/969/saison_id/2021',
 'https://www.transfermarkt.co.uk/stade-reims/startseite/verein/1421/saison_id/2021',
 'https://www.transfermarkt.co.uk/rc-strassburg-alsace/startseite/verein/667/saison_id/2021',
 'https://www.transfermarkt.co.uk/f

In [180]:
df5 = build_df(every_team_links)

In [181]:
df5 = df5.dropna()
df5

,Players,Values
0,Gianluigi Donnarumma,€50.00m
1,Keylor Navas,€8.00m
2,Sergio Rico,€4.00m
3,Alexandre Letellier,€400k
4,Garissone Innocent,€300k
...,...,...
655,Josué Casimir,€200k
656,Matthis Abline,€3.00m
657,Pape Ibnou Ba,€800k
658,Jamal Thiaré,€700k


In [182]:
df5.to_csv('L12021.csv', index=False)